In [4]:
from google.cloud import firestore
from google.oauth2 import service_account

# import firebase_admin
# default_app = firebase_admin.initialize_app()
# Path to your service account JSON file
SERVICE_ACCOUNT_FILE = "./serviceAccountKey.json"

# Initialize Firestore client with specific database
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
db = firestore.Client(project="endless-bounty-433922-g4", database="final-project-db", credentials=credentials)

In [5]:
def query_menu_items():
    try:
        # Reference the 'MenuItems' collection
        menu_ref = db.collection("MenuItems")
        # Query for items where 'name' is 'Burger'
        docs = menu_ref.where("name", "==", "Burger").stream() 
        print("Query Results:")
        for doc in docs:
            print(f"{doc.id} => {doc.to_dict()}")

    except Exception as e:
        print(f"Error querying Firestore: {e}")

# Run the query function
query_menu_items()


Query Results:
menu1 => {'availability': True, 'description': 'This is a description', 'price': 9.99, 'name': 'Burger'}


In [16]:
# Adding more data to MenuItems collection
def add_menu_items():
    items = [
        {"name": "Pizza", "description": "Delicious cheese pizza", "price": 12.99, "availability": True},
        {"name": "Pasta", "description": "Creamy Alfredo pasta", "price": 11.49, "availability": True},
        {"name": "Salad", "description": "Fresh garden salad", "price": 7.99, "availability": True}
    ]
    for i, item in enumerate(items, start=1):
        doc_ref = db.collection("MenuItems").document(f"menu{i}")
        doc_ref.set(item)
    print("Sample data added to MenuItems collection.")

# Add the sample data
add_menu_items()


Sample data added to MenuItems collection.


In [12]:
def update_menu_item(menu_id, fields_to_update):
    """
    Update specific fields of a menu item in the MenuItems collection.

    :param menu_id: The ID of the menu item to update.
    :param fields_to_update: A dictionary containing the fields and their new values.
    """
    try:
        doc_ref = db.collection("MenuItems").document(menu_id)
        doc_ref.update(fields_to_update)
        print(f"Menu item '{menu_id}' updated successfully.")
    except Exception as e:
        print(f"Error updating menu item '{menu_id}': {e}")

# Example usage
update_menu_item("menu1", {"price": 12.99, "availability": False})


Menu item 'menu1' updated successfully.


In [17]:
def delete_menu_item(menu_id):
    """
    Delete a menu item from the MenuItems collection.

    :param menu_id: The ID of the menu item to delete.
    """
    try:
        doc_ref = db.collection("MenuItems").document(menu_id)
        doc_ref.delete()
        print(f"Menu item '{menu_id}' deleted successfully.")
    except Exception as e:
        print(f"Error deleting menu item '{menu_id}': {e}")

# Example usage
delete_menu_item("menu4")


Menu item 'menu4' deleted successfully.


In [ ]:
def place_order(table_id, items):
    try:
        doc_ref = db.collection("Orders").document()
        order_data = {
            "table_id": table_id,
            "items": items,
            "status": "Pending",
            "timestamp": firestore.SERVER_TIMESTAMP
        }
        doc_ref.set(order_data)
        print("Order placed successfully!")
    except Exception as e:
        print(f"Error placing order: {e}")

# Example usage
place_order("table1", ["Burger", "Fries"])


Order placed successfully!


In [8]:
def update_order_status(order_id, new_status):
    try:
        order_ref = db.collection("Orders").document(order_id)
        order_ref.update({"status": new_status})
        print(f"Order {order_id} status updated to {new_status}.")
    except Exception as e:
        print(f"Error updating order status: {e}")

# Example usage
update_order_status("0S74p60GF130lbZHhSbi", "Preparing")


Order 0S74p60GF130lbZHhSbi status updated to Preparing.


In [ ]:
def get_orders_by_table(table_id):
    try:
        orders_ref = db.collection("Orders")
        docs = orders_ref.where("table_id", "==", table_id).stream()
        print(f"Orders for table {table_id}:")
        for doc in docs:
            print(f"{doc.id} => {doc.to_dict()}")
    except Exception as e:
        print(f"Error fetching orders: {e}")

# Example usage
get_orders_by_table("table1")


Orders for table table1:
0S74p60GF130lbZHhSbi => {'status': 'Preparing', 'tableID': 'table1', 'items': ['Burger', 'Fries'], 'timestamp': DatetimeWithNanoseconds(2024, 12, 6, 21, 59, 56, 589000, tzinfo=datetime.timezone.utc)}


In [18]:
def view_menu():
    """
    Fetch and display all menu items in the MenuItems collection.
    """
    try:
        menu_ref = db.collection("MenuItems")
        docs = menu_ref.stream()
        print("Menu Items:")
        for doc in docs:
            print(f"{doc.id} => {doc.to_dict()}")
    except Exception as e:
        print(f"Error fetching menu items: {e}")

# Example usage
view_menu()


Menu Items:
menu1 => {'availability': True, 'description': 'Delicious cheese pizza', 'price': 12.99, 'name': 'Pizza'}
menu2 => {'availability': True, 'description': 'Creamy Alfredo pasta', 'price': 11.49, 'name': 'Pasta'}
menu3 => {'availability': True, 'description': 'Fresh garden salad', 'price': 7.99, 'name': 'Salad'}


In [19]:
def view_all_orders():
    """
    Fetch and display all orders in the Orders collection.
    """
    try:
        orders_ref = db.collection("Orders")
        docs = orders_ref.stream()
        print("All Orders:")
        for doc in docs:
            print(f"{doc.id} => {doc.to_dict()}")
    except Exception as e:
        print(f"Error fetching orders: {e}")

# Example usage
view_all_orders()


All Orders:
0S74p60GF130lbZHhSbi => {'status': 'Preparing', 'tableID': 'table1', 'items': ['Burger', 'Fries'], 'timestamp': DatetimeWithNanoseconds(2024, 12, 6, 21, 59, 56, 589000, tzinfo=datetime.timezone.utc)}
order1 => {'status': 'Pending', 'tableID': '1', 'items': ['Burger', 'fries'], 'timestamp': DatetimeWithNanoseconds(2024, 11, 19, 2, 17, 36, 75000, tzinfo=datetime.timezone.utc)}


In [20]:
import qrcode

def generate_qr_code(table_id, base_url="https://restaurant.com"):
    """
    Generate a QR code for a specific table and save it as a PNG file.

    :param table_id: Unique identifier for the table (e.g., "table1").
    :param base_url: Base URL to encode in the QR code.
    """
    try:
        # Create the data to encode
        qr_code_data = f"{base_url}/{table_id}"
        
        # Generate QR code
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )
        qr.add_data(qr_code_data)
        qr.make(fit=True)

        # Create an image from the QR Code instance
        img = qr.make_image(fill_color="black", back_color="white")

        # Save the QR code image
        file_name = f"{table_id}_qr.png"
        img.save(file_name)
        print(f"QR code for {table_id} saved as {file_name}.")

        # Return the encoded data for Firestore storage
        return qr_code_data

    except Exception as e:
        print(f"Error generating QR code for {table_id}: {e}")

# Example usage
qr_data = generate_qr_code("table1")
print(f"Generated QR Code Data: {qr_data}")


QR code for table1 saved as table1_qr.png.
Generated QR Code Data: https://restaurant.com/table1


In [21]:
def store_qr_code_in_firestore(table_id, qr_code_data):
    """
    Store the QR code data and table information in the Tables collection.

    :param table_id: Unique identifier for the table.
    :param qr_code_data: Encoded data in the QR code.
    """
    try:
        doc_ref = db.collection("Tables").document(table_id)
        table_data = {
            "table_id": table_id,
            "qr_code_data": qr_code_data,
            "status": "Available"  # Default status
        }
        doc_ref.set(table_data)
        print(f"Table {table_id} data stored in Firestore.")
    except Exception as e:
        print(f"Error storing data for {table_id}: {e}")

# Example usage
store_qr_code_in_firestore("table1", qr_data)


Table table1 data stored in Firestore.


In [24]:
def generate_and_update_tables(base_url="https://restaurant.com"):
    """
    Generate QR codes for tables 1-8 and update Firestore with their data.

    :param base_url: Base URL to encode in the QR code.
    """
    for i in range(1, 9):  # Loop through tables 1 to 8
        table_id = f"table{i}"
        
        # Generate the QR code and get the encoded data
        qr_code_data = generate_qr_code(table_id, base_url)
        
        # Update the table data in Firestore
        if qr_code_data:
            store_qr_code_in_firestore(table_id, qr_code_data)
        else:
            print(f"Failed to generate QR code for {table_id}")

# Run the function
generate_and_update_tables()

QR code for table1 saved as table1_qr.png.
Table table1 data stored in Firestore.
QR code for table2 saved as table2_qr.png.
Table table2 data stored in Firestore.
QR code for table3 saved as table3_qr.png.
Table table3 data stored in Firestore.
QR code for table4 saved as table4_qr.png.
Table table4 data stored in Firestore.
QR code for table5 saved as table5_qr.png.
Table table5 data stored in Firestore.
QR code for table6 saved as table6_qr.png.
Table table6 data stored in Firestore.
QR code for table7 saved as table7_qr.png.
Table table7 data stored in Firestore.
QR code for table8 saved as table8_qr.png.
Table table8 data stored in Firestore.


In [23]:
def update_table_with_qr_url(table_id, qr_url):
    """
    Update Firestore to include the Cloud Storage URL for the QR code.

    :param table_id: ID of the table.
    :param qr_url: Public URL of the QR code in Cloud Storage.
    """
    try:
        doc_ref = db.collection("Tables").document(table_id)
        doc_ref.update({"qr_code_url": qr_url})
        print(f"Table {table_id} updated with QR code URL.")
    except Exception as e:
        print(f"Error updating table {table_id}: {e}")

# Example usage
update_table_with_qr_url("table1", "https://storage.googleapis.com/qr-codes-bucket-1660/table1_qr.png")


Table table1 updated with QR code URL.


In [25]:
def update_all_tables_with_qr_urls(bucket_name="qr-codes-bucket-1660"):
    """
    Update Firestore with the Cloud Storage URLs for QR codes for tables 1-8.

    :param bucket_name: Name of the GCP storage bucket where QR codes are stored.
    """
    for i in range(1, 9):  # Loop through table1 to table8
        table_id = f"table{i}"
        qr_url = f"https://storage.googleapis.com/{bucket_name}/{table_id}_qr.png"
        update_table_with_qr_url(table_id, qr_url)

# Run the function
update_all_tables_with_qr_urls()


Table table1 updated with QR code URL.
Table table2 updated with QR code URL.
Table table3 updated with QR code URL.
Table table4 updated with QR code URL.
Table table5 updated with QR code URL.
Table table6 updated with QR code URL.
Table table7 updated with QR code URL.
Table table8 updated with QR code URL.
